In [1]:
(ns day7)
    (require  
     '[clojure.core.matrix :as m]
     )
*ns*

Execution error (FileNotFoundException) at day7/eval3974 (REPL:2).
Could not locate clojure/core/matrix__init.class, clojure/core/matrix.clj or clojure/core/matrix.cljc on classpath.


class java.io.FileNotFoundException: 

In [2]:
(defn mem []
        (mapv read-string (-> (slurp "data/day09.txt")
(clojure.string/split #","))))
(mem)


[1102 34463338 34463338 63 1007 63 34463338 63 1005 63 53 1102 3 1 1000 109 988 209 12 9 1000 209 6 209 3 203 0 1008 1000 1 63 1005 63 65 1008 1000 2 63 1005 63 904 1008 1000 0 63 1005 63 58 4 25 104 0 99 4 0 104 0 99 4 17 104 0 99 0 0 1101 20 0 1007 1101 0 197 1022 1102 475 1 1028 1102 30 1 1008 1101 25 0 1010 1102 1 23 1009 1101 0 22 1013 1101 470 0 1029 1102 24 1 1014 1102 1 39 1005 1101 31 0 1003 1101 807 0 1026 1101 0 26 1018 1102 1 804 1027 1101 0 0 1020 1102 1 38 1017 1101 0 27 1016 1102 443 1 1024 1101 0 36 1006 1102 21 1 1015 1101 28 0 1001 1102 33 1 1019 1102 1 37 1011 1102 1 190 1023 1101 0 434 1025 1101 34 0 1004 1102 1 1 1021 1101 0 29 1012 1102 1 32 1002 1101 35 0 1000 109 30 2105 1 -7 1001 64 1 64 1105 1 199 4 187 1002 64 2 64 109 -23 2101 0 -5 63 1008 63 32 63 1005 63 225 4 205 1001 64 1 64 1105 1 225 1002 64 2 64 109 7 2102 1 -5 63 1008 63 23 63 1005 63 251 4 231 1001 64 1 64 1106 0 251 1002 64 2 64 109 -16 2101 0 2 63 1008 63 33 63 1005 63 275 1001 64 1 64 1106 0 277 

In [3]:
(defn pow [x n]
    (apply * (repeat n x)))

 (pow 10 2)

100

In [4]:
(defn opcode [{:keys [mem ip]}]
    (mem ip 0))

(defn param [state n]
    (let [{:keys [mem ip]} state]
    (mem (+ ip n) 0)))

(defn operation [opcode]
           (mod opcode 100))

(defn mode [opcode n] 
    (case (mod 
     (quot opcode 
           (pow 10 (inc n)))
     10) 0 :pos 1 :imm 2 :rel))
    

(defn paramval [state n]
    (let [raw (param state n)]
      ;;  (println raw)
    (or (case (mode (opcode state) n)
        :imm (param state n)
        :pos ((state :mem) (param state n) 0)
        :rel ((state :mem) (+ (state :base) (param state n)) 0)
        )
    0 )))

(defn targetval [state n]
    (case (mode (opcode state) n)
       :rel (+ (state :base) (param state n))
        (param state n)
        )
 )


(mode 1001 1)
;; (run mem)



:pos

In [5]:
(defn doinput [state]
    (let [
          {:keys [mem input base]} state
          mode (mode (opcode state) 1)
          param1 (targetval state 1)
          ;;_ (println (str "tar: " param1 " " mode))
          mem (assoc mem param1  (first input))
          ]
        (update (assoc state :mem mem :yield false) 
                        :ip +  2)
        ))
    


#'day7/doinput

In [6]:
(defn dooutput [state]
    (let [
          param1 (paramval state 1)
    
          ]
        (update (update state :ip + 2)
               :output conj param1 ) 
         
         )
        
        )
    

#'day7/dooutput

In [7]:
(defn dobase [state]

    (let [
        
          param1 (paramval state 1)
          ;;_ (println "xxxx")
          ;;_ (println param1)

          newstate (update (update state :ip + 2)
               :base + param1 )
          ;;_ (println newstate)] newstate
          ] newstate
        
        ))
    

#'day7/dobase

In [8]:
(defn domult [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (targetval state 3)
    
          mem (assoc (state :mem) target (* param1 param2))
          
          ]
        ;;(println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day7/domult

In [9]:
(defn doplus [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (targetval state 3)
    
          mem (assoc (state :mem) target (+ param1 param2))
          
          ]
       ;; (println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day7/doplus

In [10]:
(defn doequals [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (targetval state 3)
        res (if (= param1 param2) 1 0)   
          mem (assoc (state :mem) target res)         
          ]
       ;; (println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day7/doequals

In [11]:
(defn doless [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)
        target (targetval state 3)
        res (if (< param1 param2) 1 0)  
        mem (assoc (state :mem) target res)
          ]
        ;;(println (str target "," param1 "," param2))
        (update (assoc state :mem mem) :ip + 4)
        )
    )

#'day7/doless

In [12]:
(defn dojumptrue [state]
    (let [
        param1 (paramval state 1)
        param2 (paramval state 2)         
          ]
    (if (not (zero? param1))
        (assoc state :ip param2)     
        (update state :ip + 3)
        )
    ))

#'day7/dojumptrue

In [13]:
(defn dojumpfalse [state]
    (let [
    
        param1 (paramval state 1)
        param2 (paramval state 2)
    

          
          ]
    (if  (zero? param1)
        (assoc state :ip param2)     
        (update state :ip + 3)
        )
    ))

#'day7/dojumpfalse

In [14]:


(defn step [state] 
    ;;(println state)
    ;;(println pos)
        (let [
          oc (opcode state)
          op (operation oc)
          ]
    ;;(println oc)
              
    (case op
        1 (doplus state)
        2 (domult state)
        3 (doinput state)
        4 (dooutput state)
        5 (dojumptrue state)
        6 (dojumpfalse state)
        7 (doless state)
        8 (doequals state)
       9 (dobase state)
        99 (assoc state :halted true)
        (throw 
         (Exception. 
          (str "unknown op " operation))))))
     


#'day7/step

In [15]:
(defn init-prog [mem input]
              
        { :mem (into {}
          (map-indexed vector mem))
         :input  input
                
         :output [] :base 0 :ip 0 :halted false :yield false} 
    )
    
 
     
(defn dump-state [{:keys [halted newout input output]}]
    (
     
     "halted: " halted
         ", newout: " newout
         ", input: " input
         ", output: " output
         ))


;;(init-amps [1 2])    
     
  


#'day7/dump-state

In [16]:
(defn run [mem input]
     (let 
         [
          prog (init-prog mem input)
          
          ]
        ;; (take 5 (iterate step prog))))
        (first (filter #(:halted %)(iterate step prog)))))
         
         

  



#'day7/run

In [17]:
 (set! *print-length* 10)
;; (run (read-string "[109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]") [])
;; (run (read-string "[1102,34915192,34915192,7,4,7,99,0]") [])
;;(run (read-string "[104,1125899906842624,99]") [])
(run (mem) [1])



{:mem {0 1102, 893 889, 920 1, 558 1001, 453 64, 584 573, 487 495, 637 1002, 972 0, 519 1002, ...}, :input [1], :output [2682107844], :base 1019, :ip 903, :halted true, :yield false}

In [18]:
 (set! *print-length* 10)
;; (run (read-string "[109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]") [])
;; (run (read-string "[1102,34915192,34915192,7,4,7,99,0]") [])
;;(run (read-string "[104,1125899906842624,99]") [])
(time (run (mem) [2]))



"Elapsed time: 939.966037 msecs"


{:mem {0 1102, 893 889, 920 1, 558 1001, 453 64, 584 573, 487 495, 637 1002, 972 0, 519 1002, ...}, :input [2], :output [34738], :base 1000, :ip 921, :halted true, :yield false}